# 4. Продвинутый pandas

In [ ]:
import pandas as pd

### Задание 1

Для датафрейма <i>visit_log</i> из материалов занятия создайте столбец source_type по следующим правилам:

- если источник traffic_source равен yandex или google, то в source_type ставится organic
- для источников paid и email из России - ставим ad
- для источников paid и email не из России - ставим other
- все остальные варианты берем из traffic_source без изменений

In [ ]:
visit_log = pd.read_csv('visit_log.csv', sep = ';')

In [ ]:
def rules(row):
  traffic_source = row['traffic_source']
  region = row['region']

  if (traffic_source == 'yandex' or traffic_source == 'google'):
    return 'organic'

  elif (traffic_source == 'paid' or traffic_source == 'email') and region == 'Russia':
    return 'ad'

  elif (traffic_source == 'paid' or traffic_source == 'email') and region != 'Russia': 
    return 'other'

  else:
    return traffic_source

In [ ]:
visit_log['source_type'] = visit_log.apply(rules, axis = 1)

In [ ]:
#без ф-ии
visit_log.loc[(visit_log.traffic_source == 'yandex') | (visit_log.traffic_source == 'google'), 'source_type'] = 'organic'
visit_log.loc[((visit_log.traffic_source == 'paid') | (visit_log.traffic_source == 'email')) & (visit_log.region =='Russia'), 'source_type'] = 'ad'
visit_log.loc[((visit_log.traffic_source == 'paid') | (visit_log.traffic_source == 'email')) & (visit_log.region !='Russia'), 'source_type'] = 'other'
visit_log.loc[visit_log.source_type.isnull(), 'source_type'] = visit_log['traffic_source']
visit_log[10:20]

### Задание 2

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

- Прочитайте содержимое файла в датафрейм
- Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном

In [ ]:
url = pd.read_table('URLs.txt',sep='\s+', engine='python')
url[url.url.str.contains('\d{8}-')].head(10)

### Задание 3

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

In [ ]:
ratings = pd.read_csv('ratings.csv')

In [ ]:
#вариант 1
(ratings
 .groupby('userId')
 .agg(
     user_live=('timestamp', np.ptp),
     rating_count=('rating', 'count'),
 )
 .query('rating_count > 100')
 ['user_live']
 .astype('timedelta64[s]')
 .mean()
)

In [ ]:
#вариант 2
ratings = ratings.groupby(['userId']).agg({'rating': 'count',
                                            'timestamp': lambda x: x.max() - x.min()})
ratings[ratings.rating > 100]['timestamp'].astype('timedelta64[s]').mean()

In [ ]:
#вариант 3
ratings = pd.read_csv('ratings.csv')
ratings = ratings.groupby(['userId']).agg(['count', 'min', 'max'])

In [ ]:
def classify(row):
    return row['timestamp']['max'] - row['timestamp']['min']   

In [ ]:
ratings['lifetime'] = ratings.apply(classify, axis = 1)
ratings[ratings['rating']['count'] > 100]['lifetime'].astype('timedelta64[s]').mean()

### Задание 4

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).
Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

<i>Обратите внимание, что в процессе объединения таблиц данные не должны теряться.</i>

Дана статистика услуг перевозок клиентов компании по типам:

- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [ ]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)

In [ ]:
merged_data = client_base.merge(rzd, on = 'client_id', how = 'outer').merge(auto, on = 'client_id', how = 'outer').merge(air, on = 'client_id', how = 'outer').fillna(0)

In [ ]:
#с адресом
merged_data

In [ ]:
#без адреса
merged_data.drop(columns = 'address')